In [1]:
from datetime import datetime
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import csv
import time

In [2]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# url to scrape 
url = 'https://www.therakyatpost.com/?s=vaksin'
driver.get(url)

# Scroll to the bottom of the page and wait for a few seconds for the page to load
while True:
    # Scroll to the bottom of the page
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait for the page to load
    time.sleep(5)

    # Check if we have reached the bottom of the page
    scroll_position = driver.execute_script("return window.pageYOffset;")
    page_height = driver.execute_script("return document.body.scrollHeight;")

    # The position for the end of page, if it is reached, quit browser
    # page_height > scroll_height also work
    if (page_height > scroll_position):
        break

page_source = driver.page_source
# Close the browser  
driver.quit()

soup = BeautifulSoup(page_source, 'html.parser')

links = soup.find_all('div', attrs={'class':'post-title'})
print(f'Links we obtained: {len(links)}')

link_date = soup.find_all('li', attrs={'class':'post-date'})
print(f'link_with_date we obtained: {len(links)}')

Links we obtained: 60
link_with_date we obtained: 60


In [3]:
# Writing to a file 
with open('the_rakya_post.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    headers = ["Article_Title", "Date", "Link"]
    writer.writerow(headers)

    for link, date in zip(links, link_date):
        # in div tag, go in and search for 'a' tags
        # Ex: Warga Emas 60 Tahun & Ke Atas Digesa Ambil Suntikan Vaksin Influenza
        article_title = link.find('a').text

        # Ex: September 8, 2022 
        date = date.text 

        # Ex: 2022-09-08 00:00:00 --> 2022-09-08
        dateobj = datetime.strptime(date, '%B %d, %Y').date()

        # in a's tag, get its link
        Link = link.find('a')['href']

        writer.writerow([article_title, dateobj, Link])

    print('CSV file saved successfully')


CSV file saved successfully
